<a href="https://colab.research.google.com/github/akaul09/LanguageTranslationModel/blob/main/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install transformers
%pip install torch
%pip install torch torchvision
%pip install pandas
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.nn.utils.rnn import pad_sequence
import torch
import pandas as pd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
df = pd.read_csv('eng_-french.csv')
df = df[0:25000]
english = df["English words/sentences"]
english = list(english)
french = df["French words/sentences"]
french = list(french)

In [12]:
input_ids = []
attention_masks = []

for english_phrase, french_translation in zip(english, french):
    encoded = tokenizer(english_phrase, french_translation, return_tensors='pt', padding=True, truncation=True)
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])
    

In [13]:
max_length = max(len(seq[0]) for seq in input_ids)

input_id_padded = []
attention_masks_padded = []

for seq in input_ids:
    pad_length = max_length - len(seq[0])
    input_id_padded.append(torch.cat([seq[0], torch.tensor([tokenizer.pad_token_id] * pad_length)]))
    
for seq in attention_masks:
    pad_length = max_length - len(seq[0])
    attention_masks_padded.append(torch.cat([seq[0], torch.tensor([0] * pad_length)]))

input_id_padded = torch.stack(input_id_padded, dim=0)
attention_masks_padded = torch.stack(attention_masks_padded, dim=0)

In [14]:
labels = torch.zeros(len(english))
dataset = TensorDataset(input_id_padded, attention_masks_padded,labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)


In [21]:
num_epoch = 2
for epoch in range(num_epoch):
    model.train()
    total_training_loss = 0
    for phrase in dataloader:
        phrase = tuple(t.to('cpu') for t in phrase)
        input_id, attention_mask_train, training_labels = phrase
        output = model(torch.tensor(input_id).to(torch.int64), attention_mask=attention_mask_train)
        loss = output.loss
        if loss is not None:  
            total_training_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
        optimizer.zero_grad()
        optimizer.step()
    average_loss = total_training_loss / len(dataloader)
    print(f"Epoch {epoch + 1}/{num_epoch}, Loss: {average_loss:.4f}")

/var/folders/sq/vvldb71x5fxcjhtx6mtb7b040000gn/T/ipykernel_8443/331543109.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model(torch.tensor(input_id).to(torch.int64), attention_mask=attention_mask_train)


Epoch 1/2, Loss: 0.0000
Epoch 2/2, Loss: 0.0000


In [22]:
model.save_pretrained("fine_tuned_translation_model")
tokenizer.save_pretrained("fine_tuned_translation_model")

('fine_tuned_translation_model/tokenizer_config.json',
 'fine_tuned_translation_model/special_tokens_map.json',
 'fine_tuned_translation_model/vocab.txt',
 'fine_tuned_translation_model/added_tokens.json')

In [36]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("fine_tuned_translation_model")
model = BertForSequenceClassification.from_pretrained("fine_tuned_translation_model")


In [51]:
test_english_sentences = ["Hi.", "Translate this sentence.", "What's the weather like today?"]


In [52]:
input_ids = []
attention_masks = []

for english_sentence in test_english_sentences:
    encoded = tokenizer(english_sentence, return_tensors='pt', padding=True, truncation=True)
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])





In [53]:
max_length = max(len(seq[0]) for seq in input_ids)

input_id_padded2 = []
attention_masks_padded2 = []

for seq in input_ids:
    pad_length = max_length - len(seq[0])
    input_id_padded2.append(torch.cat([seq[0], torch.tensor([tokenizer.pad_token_id] * pad_length)]))
    
for seq in attention_masks:
    pad_length = max_length - len(seq[0])
    attention_masks_padded2.append(torch.cat([seq[0], torch.tensor([0] * pad_length)]))

input_id_padded2 = torch.stack(input_id_padded2, dim=0)
attention_masks_padded2 = torch.stack(attention_masks_padded2, dim=0)

In [54]:
attention_masks_padded2

tensor([[1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [55]:

model.eval()

with torch.no_grad():
    outputs = model(torch.tensor(input_id_padded2).to(torch.int64), attention_mask=attention_masks_padded2)

logits = outputs.logits

predicted_labels = (logits > 0).int()
predicted_french_sentences = [tokenizer.decode(ids, skip_special_tokens=True) for ids in predicted_labels]

for i, (english_sentence, french_translation) in enumerate(zip(test_english_sentences, predicted_french_sentences)):
    print(f"Input English: {english_sentence}")
    print(f"Predicted French: {french_translation}")
    print()


Input English: Hi.
Predicted French: [unused1]

Input English: Translate this sentence.
Predicted French: [unused1]

Input English: What's the weather like today?
Predicted French: [unused1]



/var/folders/sq/vvldb71x5fxcjhtx6mtb7b040000gn/T/ipykernel_8443/4115355536.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(input_id_padded2).to(torch.int64), attention_mask=attention_masks_padded2)
